In [35]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D, merge, Dropout
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

In [36]:
import preprocess2 as pp
# a képek betöltése és előfeldolgozása
csv_data, author_stat = pp.csv_load()
# a 10 legtöbb képpel rendelkező alkotót kiválogatjuk
authors_to_select = author_stat[-100:]
authors_to_select

[('358474690052f63f9107c823b7a61686', 165),
 ('342d7068656b00878b23e1031a3ddf50', 165),
 ('db85b84c1c01fa143b9294f00f7acbeb', 167),
 ('795985c5c0e7c704a354a487bdffeef5', 171),
 ('30134467c49a3350f74b4311506f8386', 171),
 ('3a320926ef941d53716127548523450c', 172),
 ('ed9cf03b71a8a71d41ecb4e8406a0877', 172),
 ('a087f33967e659c63924aa07c89b948f', 172),
 ('24ed88087b0ffeb763e89a197a5cde0d', 172),
 ('a7b6670e2d23f415998849a8fdedae18', 174),
 ('8b1800319323ab9b0ce2e0b11b3f4772', 175),
 ('56221362cba30b7e0c96111cc38c5fcd', 176),
 ('79f6b47483600c140c91e901f746a368', 176),
 ('3b4ee6d6140803ebef405983f23ffcc1', 176),
 ('83518d02afc5754dd9e79985f56c3355', 178),
 ('9c948f31fc16aca442cc91e7fe05fb95', 179),
 ('b31c8ec49c09506f17ec3c0c04fce567', 179),
 ('062244b785e58518f2f189329633c59e', 181),
 ('63e73f693e541f53d98ed6c93d03ef40', 185),
 ('39c465a1d078121873a31807ce9dc2ed', 185),
 ('ea2beb11e25ec317263983dbdd6de81e', 186),
 ('74a1dad53214d04514269a188f16683a', 187),
 ('1d6d112c140cb6d1d763a7ca73f2e

In [37]:
csv_data = pp.csv_select(csv_data, authors_to_select)
# az útvonalnak az összes képet tartalmazó könyvtárra kell mutatnia, mert különben
# nem fog tudni belőle elég képet kiolvasni
train_images, labels = pp.load_images(csv_data, "train_sample")

5 images in the result


In [38]:
# adatok megfelelő formátumra hozása a keras számára
encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[3 1 2 0 4]
[[ 0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.]]


In [39]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [40]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót


In [41]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
desired_layers = [28, 44, 60, 70, 92, 114, 136, 158, 172, 194]
style_layers = [None]*len(desired_layers)

for i in range(len(desired_layers)):
    style_layers[i] = base_model.layers[desired_layers[i]].output
    style_layers[i] = GlobalAveragePooling2D()(style_layers[i])
    style_layers[i] = Dense(base_model.layers[desired_layers[i]].output_shape[3], activation='relu')(style_layers[i])

tf
tf
tf
tf
tf
tf
tf
tf
tf
tf


In [42]:
# egymás mellé tesszük a különböző szintű feature-öket
ff = merge(style_layers, mode='concat')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(2048, activation='relu')(ff)
ff = Dropout(0.5)(ff)
ff = Dense(1024, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

In [43]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [44]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=10, validation_split=0.2, callbacks=[history])

Train on 4 samples, validate on 1 samples
Epoch 1/10
4/4 [==============================] - 4s - loss: 1.4834 - acc: 0.2500 - val_loss: 13.2023 - val_acc: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 2s - loss: 0.0941 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 2s - loss: 0.0036 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 2s - loss: 0.0114 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 2s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 8/10
4/4 [==============================] - 2s - loss: 1.1921e-07 - acc:

In [45]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

Az Inception V3 konvolúciós rétegei:
0 input_2
1 convolution2d_95
2 batchnormalization_95
3 convolution2d_96
4 batchnormalization_96
5 convolution2d_97
6 batchnormalization_97
7 maxpooling2d_4
8 convolution2d_98
9 batchnormalization_98
10 convolution2d_99
11 batchnormalization_99
12 maxpooling2d_5
13 convolution2d_103
14 batchnormalization_103
15 convolution2d_101
16 convolution2d_104
17 batchnormalization_101
18 batchnormalization_104
19 averagepooling2d_11
20 convolution2d_100
21 convolution2d_102
22 convolution2d_105
23 convolution2d_106
24 batchnormalization_100
25 batchnormalization_102
26 batchnormalization_105
27 batchnormalization_106
28 mixed0
29 convolution2d_110
30 batchnormalization_110
31 convolution2d_108
32 convolution2d_111
33 batchnormalization_108
34 batchnormalization_111
35 averagepooling2d_12
36 convolution2d_107
37 convolution2d_109
38 convolution2d_112
39 convolution2d_113
40 batchnormalization_107
41 batchnormalization_109
42 batchnormalization_112
43 batchnorma

In [46]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [47]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=10, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 4 samples, validate on 1 samples
Epoch 1/10
4/4 [==============================] - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 2/10
4/4 [==============================] - 1s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 3/10
4/4 [==============================] - 1s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 4/10
4/4 [==============================] - 1s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 5/10
4/4 [==============================] - 3s - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 6/10
4/4 [==============================] - 2s - loss: 1.9372e-07 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 7/10
4/4 [==============================] - 1s - loss: 2.9206e-06 - acc: 1.0000 - val_loss: 16.1181 - val_acc: 0.0000e+00
Epoch 8/10
4/4 [==============================] - 1s - loss: 1

In [50]:
predicted_values = model.predict(np.asarray(train_images))

In [51]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

[3, 1, 2, 0, 2]

In [52]:
encoder.inverse_transform(festok)

array(['cc47068929413a16aa707faefbdf4b70',
       '8e441c5899bf3d2f3b2c493e62fb92bf',
       'c56bcab4b317984013ebef5d3c4b5906',
       '3cc9a44380296d93e68b71a27643c25f',
       'c56bcab4b317984013ebef5d3c4b5906'], 
      dtype='<U32')

In [ ]:
# vissza kéne alakítani valahogy...
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])\
.reshape(n_samples, n_features) - ohc.feature_indices_[:-1]